In [1]:
#Toronto Neighbourhood Dataframe from Part1 of the assignment

get_ipython().system(u'pip install --upgrade beautifulsoup4')
get_ipython().system(u'pip install --upgrade lxml')
get_ipython().system(u'pip install --upgrade html5lib')
get_ipython().system(u'pip install --upgrade requests')

from bs4 import BeautifulSoup
import requests
import pandas as pd

source = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

soup = BeautifulSoup(source,'lxml')

Toronto_table = soup.find('table',{'class':'wikitable sortable'})

PostalCodeList=[]
for i in Toronto_table.findAll('tr'):
    split_rows=i.text.split('\n')
    PostalCodeList.append(split_rows)
    empty_string=''
    while empty_string in split_rows:
        split_rows.remove(empty_string) 

PostalCodeDf=pd.DataFrame(PostalCodeList[1:], columns=PostalCodeList[0])

FormattedDf=PostalCodeDf[PostalCodeDf.Borough!='Not assigned']

for i in range(0,len(FormattedDf)):
    if FormattedDf.iloc[i]['Neighbourhood']=='Not assigned':
        FormattedDf.iloc[i]['Neighbourhood']=FormattedDf.iloc[i]['Borough']
FormattedDf=FormattedDf.reset_index()
FormattedDf=FormattedDf.drop(['index'],axis=1)
GroupedDf=pd.DataFrame(FormattedDf.groupby('Postcode',as_index=False).agg(lambda x:', '.join(x['Neighbourhood'])))

FinalDf=pd.merge(GroupedDf, FormattedDf, left_on='Postcode', right_on='Postcode', how='left').drop(['Borough_x','Neighbourhood_y'], axis=1)

FinalDf=FinalDf.drop_duplicates(subset=['Postcode','Borough_y','Neighbourhood_x'])

FinalDf=FinalDf[['Postcode','Borough_y','Neighbourhood_x']]

FinalDf.columns=['Postcode','Borough','Neighbourhood']

FinalDf=FinalDf.reset_index()
FinalDf=FinalDf.drop(['index'],axis=1)
FinalDf.shape

Requirement already up-to-date: beautifulsoup4 in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages
Requirement not upgraded as not directly required: soupsieve>=1.2 in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from beautifulsoup4)
Requirement already up-to-date: lxml in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages
Requirement already up-to-date: html5lib in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages
Requirement not upgraded as not directly required: webencodings in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from html5lib)
Requirement not upgraded as not directly required: six>=1.9 in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from html5lib)
Requirement already up-to-date: requests in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages
Requirement not upgraded as not directly required: certifi>=2017.4.17 in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from requests)
Requirement not upgraded

(103, 3)

### Toronto Neighborhoods - Part 2

### Reading the cooridnates data from csv

In [2]:
import pandas as pd 

lat_long=pd.read_csv('http://cocl.us/Geospatial_data') 
lat_long.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


### Appending the coordinates from geolocation dataframe to final dataframe

In [3]:
FinalDf['Latitude']=lat_long['Latitude']
FinalDf['Longitude']=lat_long['Longitude']
FinalDf.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


### Shape of the final dataframe

In [4]:
FinalDf.shape

(103, 5)